In [ ]:
#making network info usable

In [1]:
import numpy as np
import pandas as pd
import re
import networkx as nx
import base64
import requests
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials
import fuzzywuzzy
from fuzzywuzzy import fuzz

In [26]:
#upload original csv
cover_df = pd.read_csv('/Users/oliviasato/Desktop/Network Science/covers_with_artists.csv')
#get rid of nans
cover_df = cover_df.fillna('')

#put song titles in same rows as artists
for i in range(0,len(cover_df)):
    if cover_df['Song'][i] == '':
        cover_df['Song'][i] = cover_df['Song'][i-1]
    #remove negative/lowercase problematic tracks
    if cover_df['Song'][i][0] == '-':
        cover_df['Track ID'][i] = ''
columns = ['Song', 'Track ID', 'Artist ID', 'Artist Name']
no_empties = pd.DataFrame(columns=columns)
for i in range(0,len(cover_df)):
    if not cover_df['Track ID'][i] == '':
        row_to_add = pd.DataFrame(cover_df.iloc[i]).T
        no_empties = pd.concat([no_empties, row_to_add], ignore_index=True)
cover_df = no_empties

#make 2 node lists, one of unique songs and another of unique artists
artists = []
songs = []
for i in range(len(cover_df)):  
    if not (cover_df['Artist Name'][i] in artists):
        artists.append(cover_df['Artist Name'][i])
    if not (cover_df['Song'][i] in songs):
        songs.append(cover_df['Song'][i])

#seperating song IDs and adding to seperate column
song_id = []
songs_cleaned = []
for i in range(0,len(cover_df)):
    txt = cover_df['Song'][i]
    split = re.split(', ', txt)
    songid = split[0]
    title = split[-1]
    song_id.append(songid)
    songs_cleaned.append(title)
songs = songs_cleaned

for i in range(0,len(cover_df)):
    cover_df['Song'][i] = songs[i]

cover_df['Song ID'] = song_id
cover_df['Original Artist'] = len(cover_df)*[()]
cover_df['Original Year'] = len(cover_df)*[()]
#eventually add year of cover here
cover_df = cover_df[['Song ID', 'Song', 'Original Artist', 'Original Year', 'Track ID', 'Artist ID', 'Artist Name']]

cover_df.to_csv('cover_data.csv', index=False)

In [27]:
#further modifying df
cover_df = pd.read_csv('cover_data.csv')

#take only second song id when there are two
id2 = []
for i in range(0,len(cover_df)):
    txt = cover_df['Song ID'][i]
    split = re.split(',', txt)
    second = split[-1]
    id2.append(second)

for i in range(0,len(cover_df)):
    cover_df['Song ID'][i] = id2[i]
cover_df.to_csv('cover_data.csv', index=False)

In [ ]:
#SCRAPING STARTS HERE

In [27]:
#scraping info
#these are personal to my spotify developer account. Feel free to use but you can also make your own very easily w/ https://developer.spotify.com/documentation/web-api
client_id = '98fa7d752c1d472da28147c9ebd3b277'
client_secret = '98a66db9861d4939aa2036edeabd924a'

# Encode client ID and client secret
encoded = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

# Set up headers for authentication
headers = {'Authorization': f'Basic {encoded}',}

# Set up data for token request
data = {'grant_type': 'client_credentials',}

# Make the token request
response = requests.post('https://accounts.spotify.com/api/token', headers=headers, data=data)

# Get the access token from the response
access_token = response.json().get('access_token')

In [21]:
#adding empty columns to be scraped
cover_df = pd.read_csv('cover_data.csv')
cover_df['Cover Genre'] = ''
cover_df['Popularity'] = None
cover_df['Cover Date'] = None 

In [42]:
cover_df.to_csv('cover_data_artistinfoscraped.csv', index=False)

In [ ]:
#loaded here because I scraped in increments, so I kept rerunning this to check how the csv was updating
cover_df = pd.read_csv('cover_data_artistinfoscraped.csv')

In [ ]:
#time is IMPORTANT
#I got booted for a couple of hours when I skipped it. I believe limit is 200 queries a minute, which is why I broke this into 200 cover increments
import time
#upper limit actually above 80*200 (its the length of cover_df) but I did these last ones outside the loop
for j in range(0,80):
    for i in range(j*200,(j+1)*200):
        cover_artist = cover_df['Artist Name'][i]
        search_url = f"https://api.spotify.com/v1/search"
        search_params = {"q": cover_artist, "type": "artist"}
        search_headers = {"Authorization": f"Bearer {access_token}"}
        search_response = requests.get(search_url, params=search_params, headers=search_headers)
        search_results = search_response.json()
        if "artists" in search_results and "items" in search_results["artists"]:
            artists = search_results["artists"]["items"]
        if artists:
            #assume relevant artist is first to pop up
            name = artists[0]
        cover_df['Cover Genre'][i] = name["genres"]
        cover_df['Popularity'][i] = name["popularity"]
    time.sleep(10)
cover_df.to_csv('cover_data_artistinfoscraped.csv', index=False)

In [ ]:
#scraping original artist and date from Spotify
og_artist_list = []
og_date_list = []

for j in range(0,80):
    for i in range(j*200,(j+1)*200):
        song_raw = cover_df['Song'][i]
        split = re.split(r' \(', song_raw)
        song_name = split[0]
        search_url = f"https://api.spotify.com/v1/search"
        search_params = {"q": song_name, "type": "track"}
        search_headers = {"Authorization": f"Bearer {access_token}"}
        search_response = requests.get(search_url, params=search_params, headers=search_headers)
        search_results = search_response.json()
        
        columns = ['Artists','Date']
        artists_dates_df = pd.DataFrame(columns=columns)
        
        #only collect og info for first instance of song title
        if i>0 and song_raw == cover_df['Song'][i-1]:
            #to be filled later, just not wasting time
            og_date = ''
            og_artist = ''
        
        #if the query yields results, make df of all results and select the one with the earliest release date
        elif "tracks" in search_results and "items" in search_results["tracks"]:
            tracks = search_results["tracks"]["items"]
            for k in tracks:
                #spotify sometimes returns entirely different tracks from those I am searching? Weird algorithm behavior
                #I think its suggesting what it thinks we want, filter that out
                if song_name in k['name']:
                    artists = [artist["name"] for artist in k["artists"]]
                    release_date = k["album"]["release_date"]
                    track_df = pd.DataFrame({'Artists': [artists], 'Date': [release_date]})
                    artists_dates_df = pd.concat([artists_dates_df, track_df], ignore_index=True)  
            if artists_dates_df.empty == False:
                og_date = min(artists_dates_df['Date'])
                row_index = (artists_dates_df == og_date).any(axis=1).idxmax()
                og_artist = artists_dates_df['Artists'][row_index]
            
        else:
            print("Not in search results.")
            og_date = ''
            og_artist = ''

        og_artist_list.append(og_artist)
        og_date_list.append(og_date)

#MIND THE LIMITS when breaking into smaller bits, modify i+() term so that it matches lower j limit above
for i in range(0,len(og_artist_list)):
    cover_df['Original Artist'][i+0*200] = og_artist_list[i]
    cover_df['Original Year'][i+0*200] = og_date_list[i]
    
#FIX LATER: once I scrape cover date, filter by those whose cover date precedes og date and redo og date and og artist

cover_df.to_csv('with_ogs.csv', index=False)

In [5]:
#encountering problem where good tracks are not meeting requirements to retrieve release_date, so use fuzzy matching
def is_fuzzy_in_list(test, string_list, threshold=80):
    for i in string_list:
        similarity = fuzz.partial_ratio(test, i)
        if similarity >= threshold:
            return True
    return False

In [19]:
cover_df = pd.read_csv('with_coverdates.csv')

In [29]:
#get cover date
#again, choosing earliest cover date because I am seeing problems with remastered versions etc. spitting out covers after the artists died

import time

cover_date = []

for j in range(0,80):
    for i in range(j*200,(j+1)*200):
        song_raw = cover_df['Song'][i]
        split = re.split(r' \(', song_raw)
        song_name = split[0]
        cover_artist = cover_df['Artist Name'][i]
        query = f'artist:{cover_artist} track:{song_name}'
        search_url = f"https://api.spotify.com/v1/search"
        search_params = {"q": query, "type": "track"}
        search_headers = {"Authorization": f"Bearer {access_token}"}
        search_response = requests.get(search_url, params=search_params, headers=search_headers)
        search_results = search_response.json()
                 
        potential_dates = []
        
        if "tracks" in search_results and "items" in search_results["tracks"]:
            tracks = search_results["tracks"]["items"]
            if len(tracks) == 0:
                release_date = ''
            else:
                for k in tracks:
                    if song_name in k["name"]:
                        artists = [artist["name"] for artist in k["artists"]]
                        if is_fuzzy_in_list(cover_artist,artists):
                            potential_dates.append(k["album"]["release_date"])
                if len(potential_dates) > 0:
                    release_date = min(potential_dates)
                else:
                    release_date = ''
            
        else:
            release_date = ''
            
        cover_date.append(release_date)
        
    time.sleep(20)

#MIND THE LIMITS
for i in range(0,len(cover_date)):
    cover_df['Cover Date'][i+0*200] = cover_date[i]

cover_df.to_csv('with_coverdates.csv', index=False)

/var/folders/dc/rvt7flcd7qzcbq_87lq3c8rr0000gn/T/ipykernel_52984/631394392.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cover_df['Cover Date'][i+69*200] = cover_date[i]


In [33]:
#number of cover years with dates using fuzzy matching
fuzz_df = pd.read_csv('with_coverdates.csv')
counter_fuzz = 0
for i in range(0,len(fuzz_df)):
    if type(fuzz_df['Cover Date'][i]) == str:
        counter_fuzz += 1
print('fuzz = ' + str(counter_fuzz/16149))

fuzz = 0.6714966871013686


In [34]:
#three rounds of scraping
#should I split artists if multiple names?

#FIRST - ARTIST NODES LIST: cover artist genre, cover artist popularity - DONE

#SECOND - SONG NODES LIST: og artist, og year of release - DONE
    #perhaps have to double check on secondhandsongs?

#THIRD - COVER LINKS (probs from df): year of cover - DONE (first pass at 67%)

#FOURTH - CHECK (og date < cover date)